In [13]:
import pandas as pd
import mysql.connector
import plotly.express as px
import plotly.graph_objects as go

# Подключение к базе данных
db_config = {
    'host': 'localhost',
    'database': 'projectsdb',
    'user': 'root',
    'password': 'NewStrongPassword'
}

# Загрузка SQL-запроса из файла
with open("../sql_queries/9_cooperation_ranking.sql", "r", encoding="utf-8") as f:
    query = f.read()

# Получение данных
conn = mysql.connector.connect(**db_config)
df = pd.read_sql(query, conn)
conn.close()

# === Гистограмма: Количество упоминаний сфер сотрудничества ===
df_sorted = df.sort_values(by='Количество_упоминаний', ascending=False)

fig_bar = px.bar(
    df_sorted,
    x='Сфера_сотрудничества',
    y='Количество_упоминаний',
    color='Сфера_сотрудничества',
    title='Упоминания сфер сотрудничества в проектах',
    labels={'Сфера_сотрудничества': 'Сфера', 'Количество_упоминаний': 'Количество'},
)
fig_bar.update_layout(xaxis_tickangle=-45)
fig_bar.show()

# === Круговая диаграмма: Рейтинг сфер сотрудничества ===
top_n = 7
df_top = df_sorted.head(top_n).copy()
df_rest = df_sorted.iloc[top_n:].copy()
other_sum = df_rest['Количество_упоминаний'].sum()

df_other = pd.DataFrame({
    'Сфера_сотрудничества': ['Другие'],
    'Количество_упоминаний': [other_sum]
})

df_pie = pd.concat([df_top, df_other], ignore_index=True)

fig_pie = px.pie(
    df_pie,
    names='Сфера_сотрудничества',
    values='Количество_упоминаний',
    title='Рейтинг сфер сотрудничества',
    hole=0.4
)

# Убираем подписи внутри кругов
fig_pie.update_traces(textinfo='none')

# Добавляем текст рядом с секторами, показывая проценты и названия
fig_pie.update_traces(
    hoverinfo='label+percent',  # Показываем проценты при наведении
    textposition='outside',  # Тексты будут вынесены наружу
    pull=[0.1 if i == len(df_pie) - 1 else 0 for i in range(len(df_pie))],  # Делаем сектор "Другие" чуть больше
    textinfo='percent+label'  # Показываем процент и название на внешней части сектора
)

# Обновляем макет, убираем легенду
fig_pie.update_layout(
    title='Рейтинг сфер сотрудничества 2017-2024 гг.',
    showlegend=False,  # Убираем легенду
    height=500
)

fig_pie.show()

# === Визуализация по годам (stacked bar) ===
year_query = """
SELECT 
    СС.Название AS Сфера_сотрудничества,
    П.Год,
    COUNT(*) AS Количество_упоминаний
FROM 
    Проект_Сфера ПС
JOIN 
    Сфера_сотрудничества СС ON ПС.Сфера_id = СС.ID
JOIN
    Проект П ON ПС.Проект_id = П.ID
GROUP BY 
    СС.Название, П.Год
ORDER BY 
    П.Год, Количество_упоминаний DESC;
"""

conn = mysql.connector.connect(**db_config)
df_year = pd.read_sql(year_query, conn)
conn.close()

pivot_df = df_year.pivot(index='Год', columns='Сфера_сотрудничества', values='Количество_упоминаний').fillna(0)

fig_stacked = go.Figure()
for col in pivot_df.columns:
    fig_stacked.add_trace(
        go.Bar(name=col, x=pivot_df.index, y=pivot_df[col])
    )

fig_stacked.update_layout(
    barmode='stack',
    title='Упоминания сфер сотрудничества по годам',
    xaxis_title='Год',
    yaxis_title='Количество упоминаний',
    legend_title='Сфера сотрудничества'
)
fig_stacked.show()


/var/folders/v9/0cr7_35s4q7f7g0306ks9xvm0000gn/T/ipykernel_6514/2502018829.py:20: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



/var/folders/v9/0cr7_35s4q7f7g0306ks9xvm0000gn/T/ipykernel_6514/2502018829.py:97: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

